# 03 – Evaluate Reranker (MRR@10, Recall@10)

In [ ]:
import json, random, numpy as np
from sentence_transformers import CrossEncoder
model = CrossEncoder('/mnt/data/reranker_food')
rows = [json.loads(l) for l in open('/mnt/data/pairs.jsonl', encoding='utf-8')]
evalq = [json.loads(l) for l in open('/mnt/data/eval_queries.jsonl', encoding='utf-8')]
neg_pool = []
for r in rows:
    neg_pool += r['negatives']
random.shuffle(neg_pool)

def metrics(ranks, k=10):
    mrr = np.mean([1.0/(r+1) for r in ranks])
    recall = np.mean([1.0 if r < k else 0.0 for r in ranks])
    import math
    ndcgs = []
    for r in ranks:
        if r < k: ndcgs.append(1.0/np.log2(r+2))
        else: ndcgs.append(0.0)
    return mrr, float(np.mean(ndcgs)), recall

ranks = []
for i, q in enumerate(evalq):
    pos = rows[i % len(rows)]['positive']
    candidates = [pos] + neg_pool[i*20:(i+1)*20]
    pairs = [(q['query'], c) for c in candidates]
    scores = model.predict(pairs)
    order = np.argsort(-scores)
    rank_pos = list(order).index(0)
    ranks.append(rank_pos)
mrr, ndcg, r10 = metrics(ranks, k=10)
print(f'MRR@10={mrr:.3f} nDCG@10={ndcg:.3f} Recall@10={r10:.3f}')